# Experiment generation

## Setup and imports

In [ ]:
!pip install python-pptx
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 5.0 MB/s 
     |████████████████████████████████| 149 kB 48.3 MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470952 sha256=76e4d32a3d20ba44d7271b93d2eff2f1f8668382883048a24e7e13ff88abe4ed
  Stored in directory: /root/.cache/pip/wheels/b0/38/58/8530ed1681bfee42349acf166867cc9fb369517b2fce83e599
Successfully built python-pptx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
from pptx import Presentation
from pydub import AudioSegment

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/'9.66 proj'

/content/drive/MyDrive/9.66 proj


## Generate

In [ ]:
sr = 44100
silence = AudioSegment.silent(duration=1000,frame_rate=sr)
silence_5s = AudioSegment.silent(duration=5000,frame_rate=sr)

def generate_audio_file(input_filename, output_filename):
  with open(f'conditions_data/{input_filename}','rb') as infile:
    audio_list = pickle.load(infile)

  block_names = 'abcdef'
  block_headers = []
  for c in block_names:
    block_headers.append(AudioSegment.from_file(f'sents/block_{c}.wav',format='wav',frame_rate=sr) - 6)
  block_footer = AudioSegment.from_file(f'sents/end_block.wav',format='wav',frame_rate=sr) - 6

  condition_audio = AudioSegment.empty()

  for i in range(6):
    condition_audio += block_headers[i]
    for sent in audio_list[i]:
      condition_audio += silence
      sent_audio = AudioSegment.from_file(sent_to_file[sent],format='wav',frame_rate=sr)
      condition_audio += sent_audio
    condition_audio += block_footer
    condition_audio += silence_5s
  
  condition_audio.export(f'conditions/{output_filename}',format='wav')


In [ ]:
def generate_pptx(input_filename, output_filename):
  with open(f'conditions_data/{input_filename}','rb') as infile:
    selection_list = pickle.load(infile)
  
  block_names = 'ABCDEF'

  prs = Presentation('conditions/template.pptx')
  title_layout = prs.slide_layouts[5]
  
  for i in range(6):
    slide = prs.slides.add_slide(title_layout)
    title = slide.shapes.title
    title.text = f'Block {block_names[i]}'

    for j in range(len(selection_list[i])):
      sent = selection_list[i][j]
      slide = prs.slides.add_slide(title_layout)
      title = slide.shapes.title
      title.text = sent + f'\n{j+1}'
  
  prs.save(f'conditions/{output_filename}')

In [ ]:
# generate_audio_file('audio_1.pckl','audio_1.wav')
# generate_pptx('selection_1.pckl','selection_1.pptx')

In [ ]:
for i in range(20):
  i_str = str(i+1).zfill(2)
  generate_audio_file(f'audio_{i_str}.pckl',f'audio_{i_str}.wav')
  generate_pptx(f'selection_{i_str}.pckl',f'selection_{i_str}.pptx')

In [ ]:
with open('sents/sents_list.pckl','rb') as infile:
  all_sents = pickle.load(infile)
sent_to_file = {sent:f'sents/{i}.wav' for i, sent in enumerate(all_sents)}